In [9]:
%load_ext autoreload
%autoreload 2

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [10]:
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler
import gc

import sys
sys.path.append("../src")
import FE
import dataset
import models
import util
from config import read_config, updateConfig
import train_helper
import infer_helper
import loss

In [11]:
print(torch.version.cuda)

11.1


In [12]:
BaseConfig = read_config("base")
class config(BaseConfig):
    model_version = "test25-75"
    debug=False
    epochs = 5
    hidden = [400, 300, 200, 100] # [1024, 1024, 512, 256, 128, 64] sweet spot 
    batch_size = 256
    lr=5e-4
    low_q = 0.25
    high_q = 0.75
    train_folds = [0]
    use_wandb = False
    num_workers = 0
config = updateConfig(config)

Using device: cuda
Number of device: 1
Model Output Folder: ../output/test25-75/


In [13]:
config.device

device(type='cuda')

In [14]:
train, test = dataset.read_data(config)
display(train.shape, test.shape)
display(train.head())
display(test.head())

(6036000, 9)

(4024000, 7)

,id,breath_id,R,C,time_step,u_in,u_out,pressure,fold
0,1,1,20,50,0.000000,0.083334,0,5.837492,0
1,2,1,20,50,0.033652,18.383041,0,5.907794,4
2,3,1,20,50,0.067514,22.509277,0,7.876254,2
3,4,1,20,50,0.101542,22.808823,0,11.742872,0
4,5,1,20,50,0.135756,25.355850,0,12.234987,2


,id,breath_id,R,C,time_step,u_in,u_out
0,1,0,5,20,0.000000,0.000000,0
1,2,0,5,20,0.031904,7.515046,0
2,3,0,5,20,0.063827,14.651675,0
3,4,0,5,20,0.095751,21.230610,0
4,5,0,5,20,0.127644,26.320955,0


In [15]:
train["u_out"]=0

In [16]:
# y_train_fake = np.array(list(range(30))).reshape(-1, 5)
# w_train_fake = np.array([1]*5 + [0]*25).reshape(-1, 5)
# y_pred_fake = np.array([0,1,2,3,5] + [0] * 25).reshape(-1,5)
# loss.loss_fnc(torch.tensor(y_train_fake), torch.tensor(y_pred_fake), torch.tensor(w_train_fake))
# util.get_score(y_train_fake, y_pred_fake, w_train_fake)

In [17]:
train_helper.training_loop(train.copy(), config)

Fold:  0
Prepare train valid
(4828800, 9) (1207200, 9)
(4828800, 22)
training data samples, val data samples:  (60360, 80, 22) (15090, 80, 22)
Model Size: 5696501
Epoch:  0


/media/vincentwang/Backup/GBVPP/notebook/../src/train_helper.py:195: FutureWarning: Non-finite norm encountered in torch.nn.utils.clip_grad_norm_; continuing anyway. Note that the default behavior will change in a future release to error out if a non-finite total norm is encountered. At that point, setting error_if_nonfinite=false will be required to retain the old behavior.
  torch.nn.utils.clip_grad_norm_(self.model.parameters(), self.max_grad_norm)
/home/vincentwang/.local/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:129: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `opti

[100/236]  avg loss:  4.455181884765625 inst loss:  2.150235414505005
[200/236]  avg loss:  3.2499423217773438 inst loss:  1.8562780618667603
Save Model
loss:  3.0331, val_loss 5.0138, val_score 5.0139, best_val_score 5.0139
time used:  489.2443506717682
Epoch:  1


KeyboardInterrupt: 

In [10]:
gc.collect()
torch.cuda.empty_cache()

In [11]:
test_avg = infer_helper.get_test_avg(train, test, config)

Prepare train valid
(6036000, 9) (4024000, 8)
(6036000, 22)


100%|██████████████████████████████████████████████████████████████████████| 1/1 [00:37<00:00, 37.13s/it]


count    4.024000e+06
mean     9.988471e-01
std      1.597380e-03
min      9.459643e-01
25%      9.982858e-01
50%      9.991580e-01
75%      9.999946e-01
max      1.000000e+00
Name: pressure, dtype: float64
test file saved to: ../output/test25-75//submission.csv
